<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/PCA_color_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA color augmentation

PCA color augmentation은 ImageNet Classification with Deep Convolutional
Neural Networks(AlexNet)논문에 제시된 data augmentation 기법으로 이 프로젝트를 통해 PCA color augmentation의 아이디어를 이해하고, 파이썬 코드로 구현해 보고자 한다.

# PCA color augmentation 과정

PCA color augmentation으 과정은 다음과 같다.

1. 이미지의 R,G,B값에 대한 covariance matrix(공분산 행렬)을 구한다.

2. 위에서 구한 covariance matrix 의 고윳값과 고유벡터를 구한다.

3. 고윳값에 평균이 0이고 표준편차가 0.1인 가우시안분포를 기반으로 생성한 random variable, $ \alpha $을 곱한다.

4. 각 열을 고유벡터로 갖는 행렬 [p1, p2, p3]과 그에 상응하는 고유값이 순서대로 나열된(위의 $ \alpha $이 곱해진)벡터 [] dot product한다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
img = cv2.imread('/content/drive/MyDrive/워 해머-스페이스 마린.png', cv2.IMREAD_COLOR)

In [ ]:
cv2_imshow(img)

In [ ]:
img.shape

In [ ]:
# 이미지의 색상 채널 순서는 BGR임.
img_r = img.copy()
img_r[:,:,0] = 0
img_r[:,:,1] = 0
cv2_imshow(img_r)

In [ ]:
img1 = img.copy()

In [ ]:
img1 = img1 / 255.0

In [ ]:
img2 = img1.reshape(-1, 3)
img2.shape

In [ ]:
np.mean(img2, axis = 0)

In [ ]:
img3 = img2 - np.mean(img2, axis = 0)

In [ ]:
img4 = np.cov(img3, rowvar = False) # rowvar가 True일시 row가 변수 column이 관측값으로 설정.

In [ ]:
img4

In [ ]:
np.cov(img.reshape(-1, 3), rowvar = False)

In [ ]:
eig_valse_1, eig_vecs_1 = np.linalg.eigh(np.cov(img.reshape(-1, 3), rowvar = False))
print(eig_valse_1)
print(eig_vecs_1)

In [ ]:
eig_vals, eig_vecs = np.linalg.eigh(img4) # np.linalg.eigh() 함수는 실수 대칭행렬(또는 복소수 에미리미트 행렬)의 eigen value, eigen vector을 반환함.
                                          # eigen value만 반환하는 eigvalsh(), 정사각행렬에서 작동하는 eig(), eigh)()등도 있음.

In [ ]:
print(eig_vals)  # 기본 오름차순으로 정렬됨.
print(eig_vecs)  # 열 벡터임.

In [ ]:
eig_vecs

In [ ]:
idx = np.argsort(-eig_vals)
idx

In [ ]:
eig_vals[::-1].sort()
eig_vals

In [ ]:
eig_vecs = eig_vecs[:, idx]
eig_vecs

In [ ]:
m1 = np.column_stack((eig_vecs))

In [ ]:
m1

In [ ]:
m2 = np.zeros((3, 1))
alpha = np.random.normal(0, 0.1)

In [ ]:
m2[:, 0] = alpha * eig_vals[:]

In [ ]:
m2

In [ ]:
add_vect = m1@m2

In [ ]:
add_vect

In [ ]:
img.dtype

In [ ]:
for idx in range(3):   # RGB
    img.astype('float64')[..., idx] += add_vect[idx]

In [ ]:
img = np.clip(img, 0.0, 255.0)
img = img.astype(np.uint8)

In [ ]:
cv2_imshow(img)

In [ ]:
a = np.array(range(4))
a = a.reshape(2,2)
b = np.array(range(2))
b = b.reshape(2, -1)

In [ ]:
a@b

In [ ]:
a = np.array([[0, 2], [1, 3]])
a

In [ ]:
b

In [ ]:
a = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0,
                31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0,
                35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]
b = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0,
                500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0,
                700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]

In [ ]:
plt.plot(a, b, 'ro')
plt.plot(a1, b1, 'bo')
plt.plot(a2, b2, 'yo')
plt.show

In [ ]:
an = a - np.mean(a)
an = an / np.std(an)

bn = b - np.mean(b)
bn = bn / np.std(bn)

an

In [ ]:
cov = np.cov(an, bn)
cov

In [ ]:
eval, evec = np.linalg.eigh(cov)

In [ ]:
eval

In [ ]:
evec

In [ ]:
delta = np.dot(evec, eval)

In [ ]:
a1 = a + delta[0]
b1 = b + delta[1]
a1

In [ ]:
a2 = a + delta[0]*0.5
b2 = b + delta[1]*0.5

In [ ]:
alphas = np.random.normal(0, 0.1, 3)
delta = np.dot(p, alphas*lambdas)
delta = (delta*255.).astype('int8')
pca_color_image = np.maximum(np.minimum(original_image + delta, 255), 0).astype('uint8')

In [ ]:
np.cov(a, b)

In [ ]:
np.cov(a1, b1)

In [ ]:
a2